# Código redes de sentencias de la Corte Constitucional

    Autores: Andrés Felipe Patiño y Juan Carlos Rodríguez Raga
    Universidad de los Andes
    2020-20

In [ ]:
# paquetes necesarios para la extracción de sentencias
import os
import requests
import urllib.parse
from bs4 import BeautifulSoup
import re
from urllib.request import urlopen
from random import sample
# paquetes necesarios para constituir la red
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

## Extracción de sentencias

In [ ]:
# Carpeta para guardar sentencias
folder_location = './sentencias_completas'
if not os.path.exists(folder_location):
    os.mkdir(folder_location)

In [ ]:
# Listas necesarias para extraer las sentencias de la página 
años = ['92','93','94','95','96','97','98','99','00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20']
pages = ['0', '1']

In [ ]:
# Extractor de sentencias por año
urls=[]
for i in años:
    for p in pages:
        url= "https://www.corteconstitucional.gov.co/relatoria/radicador/buscar.php?ponente=&demandado=&Sentencia=&Tipo=Sentencias&proceso=&busqueda=&conector=AND&segundotema=&anios="+str(i)+"&pg="+str(p)+"&vs=0&accion=Buscar"
        urls.append(url)
links=[]
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    results = soup.find_all('td', attrs={'width': '100%'})
    for result in results:
        hipervinculo=result.find("a")
        if hipervinculo != None:
            link=hipervinculo.get("href")
            link_completo=urllib.parse.urljoin(url,link)
            links.append(link_completo)
            filename=os.path.join(folder_location,link.split('/')[-1])
            with open(filename, 'wb') as f:
                f.write(requests.get(link_completo).content)

In [ ]:
# código para extraer sólamente una muestra (debe reemplazarse en el cuadro anterior)
#muestra = sample(links,1022)
#for link in muestra:
 #   filename=os.path.join(folder_location,link.split('/')[-1])
  #  with open(filename, 'wb') as f:
   #     f.write(requests.get(link).content)

##  Extracción partes

In [ ]:
# definición de la carpeta donde se encuentran las sentencias
files = os.listdir("./sentencias_completas/")

In [ ]:
#Código para dividir las sentencias por partes según subtítulos de interés (Antecedentes; Consideraciones|Fundamentos; Decisión|Resuelve)
sentdct = {}
noencon = []
for i,file in enumerate(files):
    with open(str("./sentencias_completas/"+files[i]), encoding="Windows-1252") as f:   
        soup = BeautifulSoup(f, "html.parser")
        f.close()
        btags = soup.find_all("b")
        trbtags = str(btags).replace("\n", " ")
        htags = soup.find_all(re.compile("h[0-9]|h"))
        trhtags = str(htags).replace("\n", " ")
        etags = soup.find_all("p", class_="Estilo")
        tretags = str(etags).replace("\n", " ")
        mtags = soup.find_all("p", class_=re.compile("MsoHeading[0-9]|MsoHeading")) 
        trmtags = str(mtags).replace("\n", " ")
        ANT = re.findall("ANTECEDENTES|Antecedentes|A.N.T.E.C.E.D.E.N.T.E.S|A.n.t.e.c.e.d.e.n.t.e.s|INFORMACI.N|Informaci.n", trbtags)
        CONS = re.findall("CONSIDERACIONES|Consideraciones|FUNDAMENTO|Fundamento|F.U.N.D.A.M.E.N.T.O|F.u.n.d.a.m.e.n.t.o", trbtags)
        RES = re.findall("DECISI.N|Decisi.n|D.E.C.I.S.I...N|D.e.c.i.s.i...|RESUELVE|Resuelve|R.E.S.U.E.L.V.E|R.e.s.u.e.l.v.e", trbtags)
        ANT1 = re.findall("ANTECEDENTES|Antecedentes|A.N.T.E.C.E.D.E.N.T.E.S|A.n.t.e.c.e.d.e.n.t.e.s|INFORMACI.N|Informaci.n", trhtags)
        CONS1 = re.findall("CONSIDERACIONES|Consideraciones|FUNDAMENTO|Fundamento|F.U.N.D.A.M.E.N.T.O|F.u.n.d.a.m.e.n.t.o", trhtags)
        RES1 = re.findall("DECISI.N|Decisi.n|D.E.C.I.S.I...N|D.e.c.i.s.i...|RESUELVE|Resuelve|R.E.S.U.E.L.V.E|R.e.s.u.e.l.v.e",trhtags)
        ANT2 = re.findall("ANTECEDENTES|Antecedentes|A.N.T.E.C.E.D.E.N.T.E.S|A.n.t.e.c.e.d.e.n.t.e.s|INFORMACI.N|Informaci.n", tretags)
        CONS2 = re.findall("CONSIDERACIONES|Consideraciones|FUNDAMENTO|Fundamento|F.U.N.D.A.M.E.N.T.O|F.u.n.d.a.m.e.n.t.o", tretags)
        RES2 = re.findall("DECISI.N|Decisi.n|D.E.C.I.S.I...N|D.e.c.i.s.i...|RESUELVE|Resuelve|R.E.S.U.E.L.V.E|R.e.s.u.e.l.v.e", tretags)
        ANT3 = re.findall("ANTECEDENTES|Antecedentes|A.N.T.E.C.E.D.E.N.T.E.S|A.n.t.e.c.e.d.e.n.t.e.s|INFORMACI.N|Informaci.n", trmtags)
        CONS3 = re.findall("CONSIDERACIONES|Consideraciones|FUNDAMENTO|Fundamento|F.U.N.D.A.M.E.N.T.O|F.u.n.d.a.m.e.n.t.o", trmtags)
        RES3 = re.findall("DECISI.N|Decisi.n|D.E.C.I.S.I...N|D.e.c.i.s.i...|RESUELVE|Resuelve|R.E.S.U.E.L.V.E|R.e.s.u.e.l.v.e", trmtags)
        sent = re.findall("([CcTtSs].[0-9][0-9][0-9]-[0-9][0-9]|[CcTtSs].[0-9][0-9][0-9][Aa]-[0-9][0-9]|[CcTtSs].[0-9][0-9][0-9][0-9]-[0-9][0-9]|[CcTtSs].[0-9][0-9][0-9][0-9][Aa]-[0-9][0-9])", str(soup))
        if sent == []:
            noencon.append("identificador %d" % (i))
        else:
            nsent = str(sent[0])
            DICTU = {"id":nsent}
            ANTt = ANT + ANT1 + ANT2 + ANT3
            CONSt = CONS + CONS1 + CONS2 + CONS3
            RESt = RES + RES1 + RES2 + RES3
            RESt.reverse()
            with open("./sentencias_completas/"+files[i], mode="r") as bigfile:
                reader = bigfile.read()
                text = reader.replace("\n", " ")
                bigfile.close()
                if RESt != [] and CONSt != [] and ANTt != []:
                    for i,part in enumerate(re.split(RESt[0], text, maxsplit=1)):
                        with open("RESm_" + str(i+1), mode="w") as newfileres:
                            newfileres.write(RESt[0]+part)
                    repart1 = open("RESm_1", "r")
                    departr = open("RESm_2", "r")
                    DICTU["RES"] = (departr.read())
                    for i,part in enumerate(re.split(CONSt[0], repart1.read(), maxsplit=1)):
                        with open("CONSm_" + str(i+1), mode="w") as newfilecons:
                            newfilecons.write(CONSt[0]+part)
                    repart2 = open("CONSm_1", "r")
                    departc = open("CONSm_2", "r")
                    DICTU["CUERPO"] = (departc.read())
                    for i,part in enumerate(re.split(ANTt[0], repart2.read(), maxsplit=1)):
                        with open("ANTm_" + str(i+1), mode="w") as newfileant:
                            newfileant.write(ANTt[0]+part)
                    departh1 = open("ANTm_1", "r")
                    departh2 = open("ANTm_2", "r")
                    DICTU["ANT"] = (departh2.read())
                    DICTU["ENC"] = (departh1.read())
                elif RESt != [] and CONSt != [] and ANTt == []:
                    for i,part in enumerate(re.split(RESt[0], reader, maxsplit=1)):
                        with open("RESm_" + str(i+1), mode="w") as newfileres:
                            newfileres.write(RESt[0]+part)
                    repart1 = open("RESm_1", "r")
                    departr = open("RESm_2", "r")
                    DICTU["RES"] = (departr.read())
                    for i,part in enumerate(re.split(CONSt[0], repart1.read(), maxsplit=1)):
                        with open("CONSm_" + str(i+1), mode="w") as newfilecons:
                            newfilecons.write(CONSt[0]+part)
                    repart2 = open("CONSm_1", "r")
                    departc = open("CONSm_2", "r")
                    DICTU["CUERPO"] = (departc.read())
                    DICTU["ENC"] = (repart2.read())
                elif RESt != [] and CONSt == [] and ANTt == []:
                    for i,part in enumerate(re.split(RESt[0], reader, maxsplit=1)):
                        with open("RESm_" + str(i+1), mode="w") as newfileres:
                            newfileres.write(RESt[0]+part)
                    repart1 = open("RESm_1", "r")
                    departr = open("RESm_2", "r")
                    DICTU["RES"] = (departr.read())
                    DICTU["CUERPO"] = (repart1.read())
                elif RESt != [] and CONSt == [] and ANTt != []:
                    for i,part in enumerate(re.split(RESt[0], reader, maxsplit=1)):
                        with open("RESm_" + str(i+1), mode="w") as newfileres:
                            newfileres.write(RESt[0]+part)
                    repart1 = open("RESm_1", "r")
                    departr = open("RESm_2", "r")
                    DICTU["RES"] = (departr.read())
                    DICTU["CUERPO"] = (repart1.read())
                else:
                    print("%s_error5" % (nsent))
            sentdct['%s' % (nsent)] = DICTU

In [ ]:
print(sentdct.keys())

## Recolección de precedentes

In [ ]:
# Listas necesarias para mantener el mismo formato de años
nyears = ["1991", "1992", "1993", "1994", "1995", "1996", "1997", "1998", "1999"]
dyears = ["2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020"]

In [ ]:
# Código que extrae los precedentes de cada sentencia según la parte de interés (Cuerpo: Consideraciones|Fundamentos y su respectivos pies de página)
# ardict es el diccionario que contiene cada identificador de la sentencia (ID) con sus conexiones (precedentes)
ardict = {}
conexiones = []
ids = []
for key in sentdct.keys():
    pint1 = sentdct[key].get("CUERPO")
    pint2 = sentdct[key].get("RES")
    soupcu = BeautifulSoup(pint1, "html.parser")
    soupres = BeautifulSoup(pint2, "html.parser")
    results1 = soupcu.find_all("a")
    results2 = soupres.find_all("p", class_= "MsoFootnoteText")
    precedentes = []
    if results2 != []:
        footname1 = [] 
        footname2 = []
        clearnames = []
        resultsdef = []
        for result1 in results1:
            footname1.append(result1.get("href"))
        for foot in footname1:
            if foot != None:
                clearnames.append(foot.replace("#", ""))
        for result2 in results2:
            apart = result2.find("a")
            if apart != None:
                name = apart.get("name")
                for clearname in clearnames:
                    if name == clearname:
                        resultsdef.append(result2)
        format1c = re.findall("(C|T|S)(-| )(\d+) del [0-9][0-9] de \w+ de (\d+)", pint1)
        for f1c in format1c:
            ele = list(f1c)
            eled = []
            for e in ele: 
                if e in nyears:
                    eled.append(re.sub("19", "-", e))
                if e in dyears:
                    eled.append(re.sub("20", "-", e))
                else:
                    eled.append(e)
            if len(eled) >= 5:
                eled.pop()
            precedentes.append("".join(eled))
        format1r = re.findall("(C|T|S)(-| )(\d+) del [0-9][0-9] de \w+ de (\d+)", str(resultsdef))
        for f1r in format1r:
            ele = list(f1r)
            eled = []
            for e in ele: 
                if e in nyears:
                    eled.append(re.sub("19", "-", e))
                if e in dyears:
                    eled.append(re.sub("20", "-", e))
                else:
                    eled.append(e)
            if len(eled) >= 5:
                eled.pop()
            precedentes.append("".join(eled))
        format2c = re.findall("(C|T|S)[-| ](\d+)/(\d+)", pint1)
        for f2c in format2c:
            ele = list(f2c)
            eled = []
            for e in ele: 
                if e in nyears:
                    eled.append(re.sub("19", "-", e))
                if e in dyears:
                    eled.append(re.sub("20", "-", e))
                else:
                    eled.append(e)
            if len(eled) >= 4:
                eled.pop()
            precedentes.append("-".join(eled))
        format2r = re.findall("(C|T|S)[-| ](\d+)/(\d+)", str(resultsdef))
        for f2r in format2r:
            ele = list(f2r)
            eled = []
            for e in ele: 
                if e in nyears:
                    eled.append(re.sub("19", "", e))
                if e in dyears:
                    eled.append(re.sub("20", "", e))
                else:
                    eled.append(e)
            if len(eled) >= 4:
                eled.pop()
            precedentes.append("-".join(eled))
        format3c = re.findall("(C|T|S)[-| ](\d+) d[e|el] (\d+)", pint1)
        for f3c in format3c:
            ele = list(f3c)
            eled = []
            for e in ele: 
                if e in nyears:
                    eled.append(re.sub("19", "", e))
                if e in dyears:
                    eled.append(re.sub("20", "", e))
                else:
                    eled.append(e)
            if len(eled) >= 4:
                eled.pop()
            precedentes.append("-".join(eled))
        format3r = re.findall("(C|T|S)[-| ](\d+) d[e|el] (\d+)", str(resultsdef))
        for f3r in format3r:
            ele = list(f3r)
            eled = []
            for e in ele: 
                if e in nyears:
                    eled.append(re.sub("19", "", e))
                if e in dyears:
                    eled.append(re.sub("20", "", e))
                else:
                    eled.append(e)
            if len(eled) >= 4:
                eled.pop()
            precedentes.append("-".join(eled))
    if results2 == []:
        format1c = re.findall("(C|T|S)(-| )(\d+) del [0-9][0-9] de \w+ de (\d+)", pint1)
        for f1c in format1c:
            ele = list(f1c)
            eled = []
            for e in ele: 
                if e in nyears:
                    eled.append(re.sub("19", "-", e))
                if e in dyears:
                    eled.append(re.sub("20", "-", e))
                else:
                    eled.append(e)
            if len(eled) >= 5:
                eled.pop()
            precedentes.append("".join(eled))
        format2c = re.findall("(C|T|S)[-| ](\d+)/(\d+)", pint1)
        for f2c in format2c:
            ele = list(f2c)
            eled = []
            for e in ele: 
                if e in nyears:
                    eled.append(re.sub("19", "-", e))
                if e in dyears:
                    eled.append(re.sub("20", "-", e))
                else:
                    eled.append(e)
            if len(eled) >= 4:
                eled.pop()
            precedentes.append("-".join(eled))
        format3c = re.findall("(C|T|S)[-| ](\d+) d[e|el] (\d+)", pint1)
        for f3c in format3c:
            ele = list(f3c)
            eled = []
            for e in ele: 
                if e in nyears:
                    eled.append(re.sub("19", "", e))
                if e in dyears:
                    eled.append(re.sub("20", "", e))
                else:
                    eled.append(e)
            if len(eled) >= 4:
                eled.pop()
            precedentes.append("-".join(eled))
    ids.append(key)
    conexiones.append(precedentes)
ardict["Id"] = ids
ardict["conexiones"] = conexiones

In [ ]:
print(len(ardict["Id"]))

## Conformación de la red

### Creación de nodos y arcos

In [ ]:
#Creación del dataframe de nodos
nodos = pd.DataFrame(ardict)

In [ ]:
print(nodos)

In [ ]:
#Creación de cada conexión entre sentencias para que sea una a una
sources = []
target = []
for i in range(len(nodos)):
    conexiones = nodos["conexiones"].values[i]
    for j in range(len(conexiones)):
        sources.append(nodos["Id"].values[i])
        target.append(conexiones[j])

In [ ]:
#Creación del dataframe de arcos, donde hay una entrada para cada sentencia (source) que cita a otra (target)
arcos = pd.DataFrame(list(zip(sources, target)), 
               columns =['Source', 'Target'])
arcos.head()

In [ ]:
# Exportación a excel sin columna de index para facilitar el paso al programa Ghepi
arcos.to_excel("arcos.xlsx", index = None)
nodos.to_excel("nodos.xlsx", index = None)

In [ ]:
# Exportación a formato de separación por comas
nodos.to_csv("nodos_vf.csv")
arcos.to_csv("arcos_vf.csv")

### Red

In [ ]:
# Creación de la red dirigida, según los dataframes de nodos y arcos
G = nx.DiGraph()
for i in range(len(nodos["Id"])):
    nodo = nodos["Id"][i]
    G.add_node(nodo)

for j in range(len(arcos["Source"])):
    n1 = arcos["Source"][j]
    n2 = arcos["Target"][j]
    G.add_edge(n1,n2)

In [ ]:
# para observar el número total de arcos y nodos
print(G.number_of_edges())
print(G.number_of_nodes())

In [ ]:
# Graficación básica de la red
fig = plt.figure(figsize = (100,100))
nx.draw(G,node_size = 40,node_color = "blue",edge_color="lightgray",with_labels=False)
plt.axis("tight")

## Para hacer pruebas, no usar

#### Códigos para hacer pruebas individuales a cada sentencia

In [ ]:
sentdct = {}
noencon = []
with open(str("./sample_sentencias/"+files[19]), encoding="Windows-1252") as f:   
    soup = BeautifulSoup(f, "html.parser")
    f.close()
    btags = soup.find_all("b")
    htags = soup.find_all(re.compile("h[0-9]|h"))
    etags = soup.find_all("p", class_="Estilo")
    mtags = soup.find_all("p", class_=re.compile("MsoHeading[0-9]|MsoHeading"))    
    ANT = re.findall("ANTECEDENTES|Antecedentes|A.N.T.E.C.E.D.E.N.T.E.S|A.n.t.e.c.e.d.e.n.t.e.s|INFORMACI.N|Informaci.n", str(btags))
    CONS = re.findall("CONSIDERACIONES|Consideraciones|FUNDAMENTO|Fundamento|F.U.N.D.A.M.E.N.T.O|F.u.n.d.a.m.e.n.t.o", str(btags))
    RES = re.findall("DECISI.N|Decisi.n|D.E.C.I.S.I...N|D.e.c.i.s.i...|RESUELVE|Resuelve|R.E.S.U.E.L.V.E|R.e.s.u.e.l.v.e|R\nE.S.U.E.L.V.E|R\ne.s.u.e.l.v.e", str(btags))
    ANT1 = re.findall("ANTECEDENTES|Antecedentes|A.N.T.E.C.E.D.E.N.T.E.S|A.n.t.e.c.e.d.e.n.t.e.s|INFORMACI.N|Informaci.n", str(htags))
    CONS1 = re.findall("CONSIDERACIONES|Consideraciones|FUNDAMENTO|Fundamento|F.U.N.D.A.M.E.N.T.O|F.u.n.d.a.m.e.n.t.o", str(htags))
    RES1 = re.findall("DECISI.N|Decisi.n|D.E.C.I.S.I...N|D.e.c.i.s.i...|RESUELVE|Resuelve|R.E.S.U.E.L.V.E|R.e.s.u.e.l.v.e|R\nE.S.U.E.L.V.E|R\ne.s.u.e.l.v.e", str(htags))
    ANT2 = re.findall("ANTECEDENTES|Antecedentes|A.N.T.E.C.E.D.E.N.T.E.S|A.n.t.e.c.e.d.e.n.t.e.s|INFORMACI.N|Informaci.n", str(etags))
    CONS2 = re.findall("CONSIDERACIONES|Consideraciones|FUNDAMENTO|Fundamento|F.U.N.D.A.M.E.N.T.O|F.u.n.d.a.m.e.n.t.o", str(etags))
    RES2 = re.findall("DECISI.N|Decisi.n|D.E.C.I.S.I...N|D.e.c.i.s.i...|RESUELVE|Resuelve|R.E.S.U.E.L.V.E|R.e.s.u.e.l.v.e|R\nE.S.U.E.L.V.E|R\ne.s.u.e.l.v.e", str(etags))
    ANT3 = re.findall("ANTECEDENTES|Antecedentes|A.N.T.E.C.E.D.E.N.T.E.S|A.n.t.e.c.e.d.e.n.t.e.s|INFORMACI.N|Informaci.n", str(mtags))
    CONS3 = re.findall("CONSIDERACIONES|Consideraciones|FUNDAMENTO|Fundamento|F.U.N.D.A.M.E.N.T.O|F.u.n.d.a.m.e.n.t.o", str(mtags))
    RES3 = re.findall("DECISI.N|Decisi.n|D.E.C.I.S.I...N|D.e.c.i.s.i...|RESUELVE|Resuelve|R.E.S.U.E.L.V.E|R.e.s.u.e.l.v.e|R\nE.S.U.E.L.V.E|R\ne.s.u.e.l.v.e", str(mtags))
    sent = re.findall("([CcTtSs].[0-9][0-9][0-9]-[0-9][0-9]|[CcTtSs].[0-9][0-9][0-9][Aa]-[0-9][0-9]|[CcTtSs].[0-9][0-9][0-9][0-9]-[0-9][0-9]|[CcTtSs].[0-9][0-9][0-9][0-9][Aa]-[0-9][0-9])", str(soup))
    if sent == []:
        noencon.append("identificador %d" % (i))
    else:
        nsent = str(sent[0])
        DICTU = {"id":nsent}
        ANTt = ANT + ANT1 + ANT2 + ANT3
        CONSt = CONS + CONS1 + CONS2 + CONS3
        RESt = RES + RES1 + RES2 + RES3
        RESt.reverse()

In [ ]:
with open("./sample_sentencias/"+files[3], mode="r") as bigfile:
    reader = bigfile.read()
    bigfile.close()
    if RESt != [] and CONSt != [] and ANTt != []:
        for i,part in enumerate(re.split(RESt[0], reader, maxsplit=1)):
            with open("RESm_" + str(i+1), mode="w") as newfileres:
                newfileres.write(RESt[0]+part)
        repart1 = open("RESm_1", "r")
        departr = open("RESm_2", "r")
        DICTU["RES"] = (departr.read())
        for i,part in enumerate(re.split(CONSt[0], repart1.read(), maxsplit=1)):
            with open("CONSm_" + str(i+1), mode="w") as newfilecons:
                newfilecons.write(CONSt[0]+part)
        repart2 = open("CONSm_1", "r")
        departc = open("CONSm_2", "r")
        DICTU["CONS"] = (departc.read())
        for i,part in enumerate(re.split(ANTt[0], repart2.read(), maxsplit=1)):
             with open("ANTm_" + str(i+1), mode="w") as newfileant:
                newfileant.write(ANTt[0]+part)
        departh1 = open("ANTm_1", "r")
        departh2 = open("ANTm_2", "r")
        DICTU["ANT"] = (departh2.read())
        DICTU["ENC"] = (departh1.read())
    elif RESt != [] and CONSt != [] and ANTt == []:
        for i,part in enumerate(re.split(RESt[0], reader, maxsplit=1)):
            with open("RESm_" + str(i+1), mode="w") as newfileres:
                newfileres.write(RESt[0]+part)
        repart1 = open("RESm_1", "r")
        departr = open("RESm_2", "r")
        DICTU["RES"] = (departr.read())
        for i,part in enumerate(re.split(CONSt[0], repart1.read(), maxsplit=1)):
            with open("CONSm_" + str(i+1), mode="w") as newfilecons:
                newfilecons.write(CONSt[0]+part)
        repart2 = open("CONSm_1", "r")
        departc = open("CONSm_2", "r")
        DICTU["CONS"] = (departc.read())
        DICTU["ENC"] = (repart2.read())
    elif RESt != [] and CONSt == [] and ANTt == []:
        for i,part in enumerate(re.split(RESt[0], reader, maxsplit=1)):
            with open("RESm_" + str(i+1), mode="w") as newfileres:
                newfileres.write(RESt[0]+part)
        repart1 = open("RESm_1", "r")
        departr = open("RESm_2", "r")
        DICTU["RES"] = (departr.read())
        DICTU["CUERPO"] = (repart1.read())
    elif RESt != [] and CONSt == [] and ANTt != []:
        print("%s_error4" % (nsent))
    else:
        print("%s_error5" % (nsent))
sentdct['%s' % (nsent)] = DICTU

In [ ]:
ardict = {}
pint1 = sentdct["C-270-93"].get("CUERPO")
pint2 = sentdct["C-270-93"].get("RES")
soupcu = BeautifulSoup(pint1, "html.parser")
soupres = BeautifulSoup(pint2, "html.parser")
results1 = soupcu.find_all("a")
results2 = soupres.find_all("p", class_= "MsoFootnoteText")
precedentes = []
if results2 != []:
    footname1 = [] 
    footname2 = []
    clearnames = []
    resultsdef = []
    for result1 in results1:
        footname1.append(result1.get("href"))
    for foot in footname1:
        if foot != None:
            clearnames.append(foot.replace("#", ""))
    for result2 in results2:
        apart = result2.find("a")
        if apart != None:
            name = apart.get("name")
            for clearname in clearnames:
                if name == clearname:
                    resultsdef.append(result2)
    format1c = re.findall("(C|T|S)(-| )(\d+) del [0-9][0-9] de \w+ de (\d+)", pint1)
    for f1c in format1c:
        ele = list(f1c)
        eled = []
        for e in ele: 
            if e in nyears:
                eled.append(re.sub("19", "-", e))
            if e in dyears:
                eled.append(re.sub("20", "-", e))
            else:
                eled.append(e)
        if len(eled) >= 5:
            eled.pop()
        precedentes.append("".join(eledc))
    format1r = re.findall("(C|T|S)(-| )(\d+) del [0-9][0-9] de \w+ de (\d+)", str(resultsdef))
    for f1r in format1r:
        ele = list(f1r)
        eled = []
        for e in ele: 
            if e in nyears:
                eled.append(re.sub("19", "-", e))
            if e in dyears:
                eled.append(re.sub("20", "-", e))
            else:
                eled.append(e)
        if len(eled) >= 5:
            eled.pop()
        precedentes.append("".join(eledr))
    format2c = re.findall("(C|T|S)[-| ](\d+)/(\d+)", pint1)
    for f2c in format2c:
        ele = list(f2c)
        eled = []
        for e in ele: 
            if e in nyears:
                eled.append(re.sub("19", "-", e))
            if e in dyears:
                eled.append(re.sub("20", "-", e))
            else:
                eled.append(e)
        if len(eled) >= 4:
            eled.pop()
        precedentes.append("-".join(eled))
    format2r = re.findall("(C|T|S)[-| ](\d+)/(\d+)", str(resultsdef))
    for f2r in format2r:
        ele = list(f2r)
        eled = []
        for e in ele: 
            if e in nyears:
                eled.append(re.sub("19", "", e))
            if e in dyears:
                eled.append(re.sub("20", "", e))
            else:
                eled.append(e)
        if len(eled) >= 4:
            eled.pop()
        precedentes.append("-".join(eled))
    format3c = re.findall("(C|T|S)[-| ](\d+) d[e|el] (\d+)", pint1)
    for f3c in format3c:
        ele = list(f3c)
        eled = []
        for e in ele: 
            if e in nyears:
                eled.append(re.sub("19", "", e))
            if e in dyears:
                eled.append(re.sub("20", "", e))
            else:
                eled.append(e)
        if len(eled) >= 4:
            eled.pop()
        precedentes.append("-".join(eled))
    format3r = re.findall("(C|T|S)[-| ](\d+) d[e|el] (\d+)", str(resultsdef))
    for f3r in format3r:
        ele = list(f3r)
        eled = []
        for e in ele: 
            if e in nyears:
                eled.append(re.sub("19", "", e))
            if e in dyears:
                eled.append(re.sub("20", "", e))
            else:
                eled.append(e)
        if len(eled) >= 4:
            eled.pop()
        precedentes.append("-".join(eled))
if results2 == []:
    format1c = re.findall("(C|T|S)(-| )(\d+) del [0-9][0-9] de \w+ de (\d+)", pint1)
    for f1c in format1c:
        ele = list(f1c)
        eled = []
        for e in ele: 
            if e in nyears:
                eled.append(re.sub("19", "-", e))
            if e in dyears:
                eled.append(re.sub("20", "-", e))
            else:
                eled.append(e)
        if len(eled) >= 5:
            eled.pop()
        precedentes.append("".join(eledc))
    format2c = re.findall("(C|T|S)[-| ](\d+)/(\d+)", pint1)
    for f2c in format2c:
        ele = list(f2c)
        eled = []
        for e in ele: 
            if e in nyears:
                eled.append(re.sub("19", "-", e))
            if e in dyears:
                eled.append(re.sub("20", "-", e))
            else:
                eled.append(e)
        if len(eled) >= 4:
            eled.pop()
        precedentes.append("-".join(eled))
    format3c = re.findall("(C|T|S)[-| ](\d+) d[e|el] (\d+)", pint1)
    for f3c in format3c:
        ele = list(f3c)
        eled = []
        for e in ele: 
            if e in nyears:
                eled.append(re.sub("19", "", e))
            if e in dyears:
                eled.append(re.sub("20", "", e))
            else:
                eled.append(e)
        if len(eled) >= 4:
            eled.pop()
        precedentes.append("-".join(eled))
ardict["sent"] = precedentes